In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
import fitz
import pandas as pd


# In[2]:

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [31]:
def extract_images_and_save_to_dataframe(input_folder, output_root_folder):
    """
    指定したフォルダ内の複数のPDFファイルから画像と座標を抽出し、データフレームに保存する関数。

    Args:
        input_folder (str): PDFファイルが格納されているフォルダへのパス。
        output_root_folder (str): 画像とデータフレームを保存するルートフォルダへのパス。
    """
    # データフレームのためのリストを初期化
    data = []

    os.makedirs(output_root_folder, exist_ok=True)

    pdf_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_document = fitz.open(pdf_file)
        pdf_filename = os.path.basename(pdf_file)

        for page_number in range(len(pdf_document)):
            page = pdf_document[page_number]

            image_infos = page.get_image_info(xrefs=True)  # 画像の情報を取得

            for img_index, img_info in enumerate(image_infos):
                # 位置情報の取得
                x0, y0, x1, y1 = img_info['bbox']

                # データフレームに情報を追加
                data.append({
                    'PDFファイル': pdf_filename,
                    'ページ番号': page_number + 1,
                    '画像インデックス': img_index,
                    'x0': x0,
                    'y0': y0,
                    'x1': x1,
                    'y1': y1
                })

        pdf_document.close()

    # データフレームに情報を格納
    df = pd.DataFrame(data)

    # データフレームをCSVファイルとして保存
    csv_filename = 'image_info.csv'
    csv_path = os.path.join(output_root_folder, csv_filename)
    df.to_csv(csv_path, index=False)
    return df

In [32]:
input_folder = '../sample-pdf'
output_root_folder = 'output-images'
df = extract_images_and_save_to_dataframe(input_folder, output_root_folder)

In [33]:
df

,PDFファイル,ページ番号,画像インデックス,x0,y0,x1,y1
0,sample1.pdf,1,0,85.050079,367.223755,510.250092,650.673767
1,sample1.pdf,2,0,85.050079,317.048981,510.250092,533.448975
2,sample1.pdf,3,0,85.050079,196.849609,483.050079,516.849609
3,sample1.pdf,4,0,85.050079,365.448669,510.250092,525.948669
4,sample1.pdf,4,1,85.050079,596.248413,510.250092,754.898438
5,sample1.pdf,5,0,85.050079,397.848694,510.250092,587.548706
6,sample1.pdf,6,0,85.050079,222.249512,478.050079,529.249512
7,sample1.pdf,7,0,85.050079,415.023682,510.250092,698.473694
8,sample2.pdf,1,0,85.050079,288.024841,510.250092,571.474854
9,sample2.pdf,2,0,85.050079,297.674591,510.250092,652.024597
